In [7]:
import os
print(os.getcwd())


C:\Users\praka\OneDrive\Documents\Coding\disease-prediction-ml\notebooks


In [8]:
import pandas as pd
import numpy as np

data = pd.read_csv('../data/diabetes.csv')

# Mark zeros as NaN for relevant features
cols = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
data[cols] = data[cols].replace(0, np.nan)

# Fill missing values with median by Outcome
for col in cols:
    for outcome in [0, 1]:
        median = data[data['Outcome']==outcome][col].median()
        data.loc[(data['Outcome']==outcome) & (data[col].isnull()), col] = median


In [9]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

X = data.drop('Outcome', axis=1)
y = data['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)


In [13]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

scaler = StandardScaler()

# Fit and transform training set, but keep column names
X_train_bal = pd.DataFrame(scaler.fit_transform(X_train_bal), columns=X_train_bal.columns)

# Transform test set, keep column names
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_train_bal.columns)


AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [ ]:
import lightgbm as lgb
from sklearn.metrics import classification_report, confusion_matrix

lgb_clf = lgb.LGBMClassifier(random_state=42)
lgb_clf.fit(X_train_bal, y_train_bal)

y_pred = lgb_clf.predict(X_test)
print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.metrics import precision_recall_curve
import numpy as np

y_proba = lgb_clf.predict_proba(X_test)[:,1]
prec, rec, thresh = precision_recall_curve(y_test, y_proba)
f1s = 2 * (prec * rec) / (prec + rec + 1e-6)
best_idx = np.argmax(f1s)
best_thresh = thresh[best_idx]

print("Best threshold:", best_thresh)
y_pred_thresh = (y_proba >= best_thresh).astype(int)
print(classification_report(y_test, y_pred_thresh))
